In [2]:
import pandas as pd
import psycopg2

In [6]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()

cur.execute("""SELECT ElementValue FROM programvariables WHERE ElementName = 'qualtrics_api_token'""")
tokenList = cur.fetchone()
conn.commit()
apiKey = tokenList[0]
apiKey


'egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF'

In [4]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()

cur.execute("""
            SELECT * FROM registreeworkshops WHERE checkedin = True AND registered = False;
            """)
conn.commit()
rows = cur.fetchall()

cur.close()
conn.close()

In [6]:
rando = pd.DataFrame(data=rows, columns=['RegID', 'WorkshopID', 'Registered', 'CheckedIn'])

In [10]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()

emails = []

for _,row in rando.iterrows():
        cur.execute("""
            SELECT Email FROM registreeInfo WHERE RegID = %s;
            """, (row.iloc[0],))
        conn.commit()
        rows = cur.fetchone()
        emails.append(rows[0])


cur.close()
conn.close()

In [19]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()
cur.execute("""
                SELECT workshops.WorkshopID, workshops.SeriesID, series.ZoomMeetingID, series.StartTime, series.EndTime FROM workshops
                JOIN series on workshops.SeriesID = series.SeriesID
                WHERE workshops.Workshopdate = now()::date;
                """)
workshopList = cur.fetchall()
conn.commit()

In [20]:
series = pd.DataFrame(data=workshopList, columns=['WorkshopID', 'SeriesID', 'ZoomID', 'StartTime', 'EndTime'])

In [2]:
import psycopg2
from datetime import datetime

In [6]:
time = datetime(2024, 4, 1, 14, 1, 30)
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()
cur.execute("""SELECT DISTINCT SeriesID FROM workshops 
            WHERE workshopDate = %s::date
            """, (time,))
match = cur.fetchall()
conn.commit()
cur.close()
conn.close()

In [8]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    database="DataLab",
    user="postgres",
    host='localhost',
    password="",
    port=5432
)

# Create a cursor object
cur = conn.cursor()

# Read the CSV file containing the names
names_df = pd.read_csv("./Unregistered.csv")

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['First Name', 'Last Name', 'Email'])

# Iterate over each row in the DataFrame
for _, row in names_df.iterrows():
    first_name = row['First Name']
    last_name = row['Last Name']
    email = row['Email']

    # Execute the SELECT query for the current name
    cur.execute("""
        SELECT ri.FirstName, ri.LastName, ri.Email
        FROM RegistreeInfo as ri
        JOIN RegistreeWorkshops as rw ON ri.RegID = rw.RegID
        WHERE (rw.CheckedIn = TRUE OR rw.Registered = TRUE)
        AND (ri.FirstName = %s AND ri.LastName = %s)
    """, (first_name, last_name))

    # Fetch all rows from the result set
    rows = cur.fetchall()

    # Create a DataFrame from the fetched rows
    df = pd.DataFrame(rows, columns=['First Name', 'Last Name', 'Email'])

    # Concatenate the DataFrame to the results DataFrame
    results_df = pd.concat([results_df, df])

# Write the results DataFrame to a CSV file
results_df.to_csv("./Duplicates.csv", index=False)

# Close the cursor and connection
cur.close()
conn.close()


In [10]:
from datetime import datetime, timezone, timedelta

def are_dates_equal(arizona_date_str, utc_date_str):
    arizona_date = datetime.strptime(arizona_date_str, "%Y-%m-%d")
    utc_date = datetime.strptime(utc_date_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
    arizona_to_utc_offset = timedelta(hours=7)  # Arizona is UTC-7
    arizona_date_utc = utc_date - arizona_to_utc_offset
    return arizona_date_utc.date() == utc_date.date()

# Test the function
arizona_date_str = "2024-04-23"
utc_date_str = "2024-04-23T20:59:07Z"
print(are_dates_equal(arizona_date_str, utc_date_str))


2024-04-23 00:00:00
True
